In [1]:
import wfdb
import numpy as np

In [2]:

# # List available records in the MIMIC database
# records = wfdb.get_record_list('mimicdb')
# print("Available Records:", records)


In [3]:
# wfdb.io.dl_database(
#     db_dir='mimicdb',
#     dl_dir='./mimic_subset',
#     records = ['037/037','208/208','041/041','055/055','039/039'],  # Directory to save the file # List of record names to download
#     overwrite=False  # Set to True to overwrite the downloaded files

# )

In [4]:
recordinfo = {'mimic_subset/209/20900001':'ii','mimic_subset/039/03900001':'iii','mimic_subset/041/04100001':'iii',
              'mimic_subset/055/05500001':'mcl1','mimic_subset/208/20800001':'ii'}
#data preparation
ecg_signals = {}
for key,value in recordinfo.items():
    record_path = key
    signal_name = value
    record = wfdb.rdrecord(record_path)
    # Display metadata
    #print("Signal Names:", record.sig_name)
    #print("Sampling Frequency:", record.fs)
    #print("Number of Samples:", len(record.p_signal))

    duration =  10 # Duration of the ECG signal to extract (in seconds)
    num_samples = duration*record.fs
    num_samples = 2**(int(np.floor(np.log2(num_samples))))
    # Extract ECG data (assuming it's labeled 'ECG')
    ecg_signal = [
        record.p_signal[:num_samples, i]
        for i, name in enumerate(record.sig_name)
        if signal_name in name.lower()
    ]
    #print(f"Extracted {len(ecg_signal)} ECG signals.")
    ecg_signals[record_path] = ecg_signal


print(ecg_signals)


{'mimic_subset/209/20900001': [array([-0.00484653, -0.03231018, -0.04038772, ...,  0.00323102,
        0.00323102,  0.00323102])], 'mimic_subset/039/03900001': [array([-0.0325, -0.0385, -0.0405, ...,  0.087 , -0.0345, -0.084 ])], 'mimic_subset/041/04100001': [array([-0.0295, -0.0295, -0.026 , ..., -0.022 , -0.0235, -0.028 ])], 'mimic_subset/055/05500001': [array([0.1375, 0.158 , 0.1315, ..., 0.003 , 0.005 , 0.0105])], 'mimic_subset/208/20800001': [array([-0.08605341, -0.09495549, -0.07863501, ..., -0.02967359,
       -0.02077151, -0.02967359])]}


In [5]:
for key,value in ecg_signals.items():
    contains_nan = np.isnan(value).any()
    print("Contains NaN:", contains_nan)  # Output: True

Contains NaN: False
Contains NaN: False
Contains NaN: False
Contains NaN: False
Contains NaN: False


In [6]:
from sklearn.metrics import mean_squared_error
import numpy as np
import time
from sklearn.metrics import mean_squared_error
from memory_profiler import memory_usage


In [7]:
from AI_FFT_no_prompt import fft_v1, fft_v2, fft_v3
from AI_FFT_prompt import fft_prompt_v1, fft_prompt_v2, fft_prompt_v3
from manual_FFT import manual_fft
from baseline_fft import baseline_fft

# Define your FFT functions in a dictionary
fft_functions = {
    "fft_v1": fft_v1,
    "fft_v2": fft_v2,
    "fft_v3": fft_v3,
    "fft_prompt_v1": fft_prompt_v1,
    "fft_prompt_v2": fft_prompt_v2,
    "fft_prompt_v3": fft_prompt_v3,
    "manual_fft": manual_fft,
}

# Initialize a results dictionary
evaluation_results = {func_name: {"accuracy": [], "runtime": [], "memory": []} for func_name in fft_functions.keys()}
baseline_runtime = []

# Evaluation Framework
for signal_ID, signal in ecg_signals.items():
    if not isinstance(signal, np.ndarray):
        signal_numpy = np.asarray(signal, dtype=float).flatten()
    
    # Evaluate runtime for baseline FFT
    start_time = time.time()
    print("start_time",start_time)
    baseline_fft_result = baseline_fft(signal_numpy)
    end_time = time.time()
    print("end_time",end_time)
    baseline_runtime.append(end_time - start_time)
    
    for func_name, fft_function in fft_functions.items():
        # Evaluate memory usage
        mem_usage = memory_usage((fft_function, (signal_numpy,)), interval=0.1, timeout=1)
        memory = max(mem_usage) - min(mem_usage)
        mem_usage.clear()  # Reset memory usage list

        # Evaluate runtime
        start_time = time.time()
        generated_fft = fft_function(signal_numpy)
        end_time = time.time()
        runtime = end_time - start_time



        # Evaluate accuracy
        mse = mean_squared_error(np.abs(generated_fft), np.abs(baseline_fft_result))

        # Store results
        evaluation_results[func_name]["accuracy"].append(mse)
        evaluation_results[func_name]["runtime"].append(runtime)
        evaluation_results[func_name]["memory"].append(memory)
    break   
# Compute average results
average_results = {}
for func_name, metrics in evaluation_results.items():
    average_results[func_name] = {
        "avg_accuracy": np.mean(metrics["accuracy"]),
        "avg_runtime": np.mean(metrics["runtime"]),
        "avg_memory": np.mean(metrics["memory"])
    }

# Compute average runtime for baseline FFT
average_baseline_runtime = np.mean(baseline_runtime)

# Print results
print("Average Evaluation Results:")
for func_name, averages in average_results.items():
    print(f"{func_name}: Accuracy (MSE): {averages['avg_accuracy']:.5f}, "
          f"Runtime: {averages['avg_runtime']:.5f}s, "
          f"Memory: {averages['avg_memory']} MB")

print(f"Baseline FFT: Runtime: {average_baseline_runtime:}s")

NameError: name 'profile' is not defined